In [4]:
import numpy as np
import os
def dp_matching(feature_1, feature_2):
    '''
    DP Matching을 수행한다
    입력:
        feature_1: 비교할 특징값 배열 1
        feature_2: 비교한 특징값 배열 2
    출력:
        total_cost: 최단 경로의 총 비용
        min_path: 최단 경로 프레임 대응
        
    '''
    # 프레임 수와 차원 수를 추출
    (n_frames_1, num_dims) = np.shape(feature_1)
    n_frames_2 = np.shape(feature_2)[0]
    
    # 거리 (비용) 행렬을 계산한다
    distance = np.zeors((n_frames_1, n_frames_2))
    for n in range(nframes_1):
        for m in range(nframes_2):
            # feature_1의 n 번째 프레임과
            # feature_2의 m 번째 프레임의
            # 유클리드 거리 제곱을 계산
            distance[n,m] = np.sum((feature_1[n] - feature_2[m])**2)
            
    
    # 누적 비용 행렬
    cost = np.zeros((nframes_1, n_frames_2))
    #이동 종류 (세로/사선/가로) 를 기록하는 행렬
    # 0: 세로 이동, 1: 사선 이동, 2: 가로이동
    track = np.zeros((nframes_1, nframes_2), np.int16)
    
    # 시작지점의 거리
    cost[0,0] = distance[0, 0]
    
    # 0번째 행: 반드시 세로로 (아래로) 이동한다
    for n in range(1, nframes_1):
        cost[n,0] = cost[n-1,0]+distance[n,0]
        track[n,0] = 0
    
    # 0번째 열: 반드시 가로로 (우측으로) 이동한다
    for n in range(1, n_frames_2):
        cost[0,m] = cost[0, m-1] + distance[0, m]
        track[0, m] = 2
        
    # 그외 : 가로 세로 사선 중에 최소 비용으로 이동한다.
    for n in range(1, nframes_1):
        for m in range(1, nframes_2):
            # 세로로 이동했을때 누적 비용
            vertical = cost[n-1,m] + distance[n, m]
            # 사선으로 이동했을 때 누적 비용
            # (사선은 2배 가중치를 부여함)
            diagonal = cost[n-1,m-1]+2*distance[n,m]
            # 가로로 이동했을 때 누적 비용
            horizontal = cost[n,m-1] + distance[n,m]
            
            # 누적 비용이 최소인 이동 경로를 선택한다
            candidate = [vertical, diagonal, horizontal]
            transition = np.argmin(candidate)
            
            # 누적 비용과 이동 방향을 기록한다
            cost[n,m] = candidate[transition]
            track[n,m] = transition
            
        
    #총 비용은 cost 행렬의 최종행 X 최종 열 값
    #특징값은 프레임수로 정규화
    total_cost = cost[-1,-1]/(nframes_1,n_frames_2)
    
    #Back Track
    # 끝에서 track 값을 기준으로 역추적하여
    #최소비용 경로를 구한다
    min_path= []
    n= nframes_1 -1
    m= nframes_2 -1
    while True:
        #현재 프레임 위치를 min_path 에 추가한다
        min_path.append([n,m])
        
        # 시작 지점에 도달하면 종료한다
        if n==0 and m==0:
            break
        # track 값을 확인한다
        if track[n,m] ==0:
            #세로 이동일 경우
            n-=1
        elif track[n,m] ==1:
            # 사선 이동일 경우
            n -=1
            m -=1
        else:
            #가로 이동일 경우
            m-=1
    #min_path 를 역순으로 교체한다
    min_path= min_path[::-1]
    
    # 총비용과 최종 경로를 출력한다
    return total_cost, min_path

if __name__ == "__main__":
    # 읽어올 mfcc 파일경로
    mfcc_file_1 = './mfcc/REPEAT500_set1_009.bin'
    mfcc_file_2 = './mfcc/REPEAT500_set2_009.bin'
    
    result = './alignment.txt'
    
    #MFCC 차원수
    num_dims = 13
    #특징값 데이터를 특징값 파일에서 읽어온다
    mfcc_1 = np.fromfile(mfcc_file_1, dtype=np.float32)
    mfcc_2 = np.fromfile(mfcc_file_2, dtype=np.float32)
    # 프레임 수 * 차원수 행렬로 변환
    mfcc_1 = mfcc_1.reshape(-1, num_dims)
    mfcc_2 = mfcc_2.reshape(-1, num_dims)
    
    #DP Matching 실행
    total_cost, min_path = dp_matching(mfcc_1, mfcc_2)
    
    #얼라이먼트 (프레임 대응) 를 파일에  쓴다
    with open(result, mode='w') as f:
        for p in min_path:
            f.write("%d %d\n" % (p[0], p[1]))

FileNotFoundError: [Errno 2] No such file or directory: './mfcc/REPEAT500_set1_009.bin'